In [1]:
import argparse
import uproot
import glob
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
tree = "Delphes"
branches = ["Track.PID", "Track.Charge", "Track.PT", "Track.P", "Track.Eta", "Track.Phi", "Track.EtaOuter", "Track.PhiOuter", "Track.XOuter", "Track.YOuter"]
pionPID = 211 # plus/minus

In [3]:
def getDataFromRootFile(file):
    with uproot.open(file) as f:
        # load the branches
        temp = {}
        for branch in branches:
            temp[branch] = np.array(ak.flatten(f[tree][branch].array()))
        
        # selection
        cut = (abs(temp["Track.PID"])==pionPID)

        # apply selection
        for branch in branches:
            temp[branch] = temp[branch][cut]
    
    return temp

In [ ]:
# Barrel radius
h=30

def getBetaNegative(y0, R):
    yc=(h**2 + y0**2 - (h**2*y0**2)/(h**2 + y0**2) - y0**4/(h**2 + y0**2) +  \
             y0*np.sqrt(-h**2*(h**2 + y0**2)*(h**2 - 4*R**2 + y0**2))/(h**2 + y0**2))/(2*h)
    beta = np.arccos((yc-h)/R)
    return beta

def getBetaPositive(y0, R):
    yc=(h**2 + y0**2 - (h**2*y0**2)/(h**2 + y0**2) - y0**4/(h**2 + y0**2) -  \
             y0*np.sqrt(-h**2*(h**2 + y0**2)*(h**2 - 4*R**2 + y0**2))/(h**2 + y0**2))/(2*h)
    beta = np.arccos((h-yc)/R)
    return beta

def getBeta(y0, R, q):
    return np.where(q<0, getBetaNegative(y0, R), getBetaPositive(y0, R))

def getYentryNegQ(R, beta):
    yc = R*np.cos(beta)+h
    y0 = -np.sqrt(R**2 - yc**2) + np.sqrt(-h**2 + R**2 + 2*h*yc - yc**2)
    return y0
    
def getYentryPosQ(R, beta):
    yc = -R*np.cos(beta)+h
    y0 = np.sqrt(R**2 - yc**2) - np.sqrt(-h**2 + R**2 + 2*h*yc - yc**2)
    return y0
    
def getYentry(R, q, beta):
    return np.where(q<0, getYentryNegQ(R, beta), getYentryPosQ(R, beta))


def getCircleCenterPos(phi):
    return phi+np.pi/2

def getCircleCenterNeg(phi):
    return phi-np.pi/2

def getCircleCenter(x,y,R,phi,q):
    theta = np.where(q<0,getCircleCenterPos(phi),getCircleCenterNeg(phi))

    x0=x-R*np.cos(theta)
    y0=y-R*np.sin(theta)

    return x0,y0

def getPointOnCircle(x0,y0,d,R):
    x = (1/(2*(x0**2 + y0**2)))*(d**2*x0 - R**2*x0 + x0**3 + x0*y0**2 - np.sqrt(-d**4*y0**2 + 2*d**2*R**2*y0**2 - R**4*y0**2 + 2*d**2*x0**2*y0**2 + 2*R**2*x0**2*y0**2 - x0**4*y0**2 + 2*d**2*y0**4 + 2*R**2*y0**4 - 2*x0**2*y0**4 - y0**6))
        
    y = -(1/(2*y0))*(-d**2 + R**2 - x0**2 - y0**2 + (d**2*x0**2)/(x0**2 + y0**2) - (R**2*x0**2)/(x0**2 + y0**2) + x0**4/(x0**2 + y0**2) + (x0**2 y0**2)/(x0**2 + y0**2) - (1/(x0**2 + y0**2))*x0 np.sqrt(-d**4*y0**2 + 2*d**2*R**2*y0**2 - R**4*y0**2 + 2*d**2*x0**2*y0**2 + 2*R**2*x0**2*y0**2 - x0**4*y0**2 + 2*d**2*y0**4 + 2*R**2*y0**4 - 2*x0**2*y0**4 - y0**6))
    
    return x,y


def assumeCircle(temp):
    # Get particle track radius
    R = temp["Track.PT"]*5.36/(np.abs(temp["Track.Charge"])*1.60217663*3.8)*1000

    # The maximum and minimum possible entry points with respect to the whole pixel sensor:
    yentrymin=-16/2+0.08125
    yentrymax=16/2-0.08125

    # Using the max and min allowed entry points, get the max and min possible beta values
    betamin = getBeta(yentrymax, R, temp["Track.Charge"])
    betamax = getBeta(yentrymin, R, temp["Track.Charge"])

    # Randomly pick a beta within the allowed range
    beta=np.empty(len(temp["Track.PID"]))
    for i in range(len(temp["Track.PID"])):
        beta[i]=np.random.uniform(betamin[i], betamax[i])

    # Get the y entry point
    yentry=getYentry(R, temp["Track.Charge"], beta)

    x0, y0 = getCircleCenter(temp["Track.XOuter"],temp["Track.YOuter"], R, temp["Track.PhiOuter"],temp["Track.Charge"])

    xHit, yHit = getPointOnCircle(x0, y0, np.sqrt(yentry**2+R, R)

    return yentry, beta

In [5]:
files = "/home/elizahoward/cmspix28-mc-sim/minbiasDataSet3/"
file = files+"minbias_0.00_0.10_GeV.root"

In [6]:
temp = getDataFromRootFile(file)


In [7]:
temp["Track.Charge"]

array([-1,  1,  1, ...,  1,  1,  1], dtype=int32)

In [8]:
temp["Track.PID"]

array([-211,  211,  211, ...,  211,  211,  211], dtype=int32)